In [1]:
from tools.SEMIC import *

In [2]:
base_path = '/media/cocoan-gpu1/wani8T/data/SEMIC_npy/'

In [3]:
loading = load_dataset(dataset_path=base_path, subj_group_size=1, workers=12)

Train dataset Subject list : 019, 059, 039, 029, 049, 018, 058, 009, 038, 048, 028, 037, 008, 027, 056, 036, 016, 046, 055, 025, 035, 045, 015, 006, 024, 044, 034, 014, 053, 023, 033, 043, 004, 052, 042, 022, 032, 003, 051, 031, 011, 002, 021, 050, 020, 010, 001

Validation dataset Subject list : 054, 005, 041, 026, 030, 057, 013, 007, 012, 040, 047, 047



In [4]:
from tqdm import tqdm

In [ ]:
with tqdm(total=10) as pbar:
    pbar.set_description('loading')
    for _ in range(10):
        loading.load('train')
        pbar.update(1)
        print('\n')

loading:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
del loading.batchset

In [14]:
import gc

In [16]:
gc.collect()

0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import nibabel as nib
import cv2
from mlxtend.preprocessing import one_hot
import timeit
from random import shuffle
from MPC_tools import *
from layers import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


%matplotlib inline

In [ ]:
def batch_norm(data, is_train, name=None):
    return tf.layers.batch_normalization(data, training=is_train, name=name)

In [ ]:
num_classes = 2 

In [ ]:
x = tf.placeholder(tf.float32,shape=[None, 592895])
y_true = tf.placeholder(tf.float32,shape=[None,num_classes])
is_train = tf.placeholder(tf.bool)
hold_prob = tf.placeholder(tf.float32)

In [ ]:
layer_1 = tf.layers.dense(x, 512)
layer_1_bath_norm = batch_norm(layer_1, is_train, name=None)
layer_1_relu = tf.nn.relu(layer_1_bath_norm)

In [ ]:
layer_2 = tf.layers.dense(layer_1_bath_norm, 256)
layer_2_bath_norm = batch_norm(layer_2, is_train, name=None)
layer_2_relu = tf.nn.relu(layer_2_bath_norm)

In [ ]:
layer_3 = tf.layers.dense(layer_2_bath_norm, 128)
layer_3_bath_norm = batch_norm(layer_3, is_train, name=None)
layer_3_relu = tf.nn.relu(layer_3_bath_norm)

In [ ]:
layer_4 = tf.layers.dense(layer_3_relu, 2)
layer_4 = tf.nn.sigmoid(layer_4)

In [ ]:
y_pred = layer_4

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
learning_rate = 1e-5
optimizer = tf.train.AdamOptimizer(learning_rate)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train = optimizer.minimize(cross_entropy)

In [ ]:
step_size = 100

In [ ]:
train_acc_list = []
train_entrop_list = []

test_acc_list = []
test_entrop_list = []

validation_acc_list = []
validataion_entrop_list = []

step_list = []

lowest_loss = None

X = []
Y = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    subject_num = 1
    start = timeit.default_timer()

    for i in range(step_size):
        
        #Loading training and validation data sets
        if i%36 == 0 and i < step_size-44:
            del X
            del Y
            
            X, Y = load_data(subject_num, shuffle=True, nan_to_num=True, one_hot_y=True, flatten_X=True)
            
            if subject_num<57:
                subject_num +=1   
            
            elif subject_num>=57:
                subject_num =1    

        #Load test data sets
        elif i == step_size-44:    
            validate_subjec_num = 58
            X, Y = load_data(validate_subjec_num, shuffle=True, nan_to_num=True, one_hot_y=True, flatten_X=True)

        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
        acc = tf.reduce_mean(tf.cast(matches,tf.float32))
        
        #training model
        if i < step_size-44:
            j = i%44
            _, train_entropy, train_accuracy = sess.run([train, cross_entropy, acc], 
                                                        feed_dict={x: X[j*100:j*100+100], 
                                                                   y_true: Y[j*100:j*100+100], 
                                                                   hold_prob: 0.5, is_train:True})

            validation_entropy, validation_accuracy= sess.run([cross_entropy, acc],
                                                  feed_dict={x: X[4400:4500], 
                                                             y_true: Y[4400:4500], 
                                                             hold_prob:1.0, is_train:False})
            if i%10 == 0:
                print("* * * * * * * * * * * * *")
                print('Currently on step {}'.format(i))
                print('##Train Cross entropy is: {}'.format(train_entropy))
                print('##Train Accuracy is: {}'.format(train_accuracy))
                print('\n')
                print('##Validation Cross entropy is: {}'.format(validation_entropy))
                print('##Validation Accuracy is: {}'.format(validation_accuracy))

                print("* * * * * * * * * * * * *")
                print('\n')
                
            train_acc_list.append(train_accuracy)
            train_entrop_list.append(train_entropy)
            
            validation_acc_list.append(validation_accuracy)
            validataion_entrop_list.append(validation_entropy)
            
            step_list.append(i)

        #testing model
        elif i > step_size-44:
            j = i%44
            test_entropy, test_accuracy= sess.run([cross_entropy, acc],
                                                  feed_dict={x:X[j*100:j*100+100], 
                                                             y_true:Y[j*100:j*100+100], 
                                                             hold_prob:1.0, is_train:False})

            test_acc_list.append(test_accuracy)
            test_entrop_list.append(test_entropy)
            
            step_list.append(i)
            
            if i%1 == 0:
                print("* * * * * * * * * * * * *")
                print('Currently on step {}'.format(i))
                print('##Test Cross entropy is: {}'.format(test_entropy))
                print('##Test Accuracy is: {}'.format(test_accuracy))

                print("* * * * * * * * * * * * *")
                print('\n')
                

        if lowest_loss == None or lowest_loss > validation_entropy:
            lowest_loss = validation_entropy
            saver.save(sess, "./model/MPC_dnn_basic_SEMIC_binary_classification_ver4.ckpt")


train_acc_list = np.array(train_acc_list)
train_entrop_list = np.array(train_entrop_list)

test_entrop_list = np.array(test_entrop_list)
test_acc_list = np.array(test_acc_list)

validation_acc_list = np.array(validation_acc_list)
validataion_entrop_list = np.array(validataion_entrop_list)

step_list = np.array(step_list)

stop = timeit.default_timer()
check_time = stop-start
now = datetime.datetime.now()

print("!!!!Finished!!!!")
print("###  Duration Time  : {} minutes   {} seconds      ###".format(int(check_time)//60, int(check_time)%60))
print("###  Excecuted Time : {}     ###".format(now))

In [ ]:
from tqdm import tqdm_notebook
import time

total = tqdm_notebook(range(100))

for n in total:
    time.sleep(0.1)